# Ensemble level 2

In [36]:
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np
import copy
from sklearn.svm import LinearSVC
from tffm import TFFMClassifier
import tensorflow as tf
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
# data analysis

In [3]:
#load or create your dataset
print('Load data...')
df_train = pd.read_csv("train.csv")
print(df_train.head())
df_test = pd.read_csv('test.csv')
ID = df_test['ID_code']
del df_test['ID_code']

# delete duplicated based on time
del df_train['ID_code']
df_train.drop_duplicates(keep='first', inplace=True)

# y_train output - deleted target value
y_train = df_train.target
del df_train['target']

# x_train
X_train = df_train.values

Load data...
   ID_code  target    var_0   var_1    var_2   var_3    var_4   var_5   var_6  \
0  train_0       0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187   
1  train_1       0  11.5006 -4.1473  13.8588  5.3890  12.3622  7.0433  5.6208   
2  train_2       0   8.6093 -2.7457  12.0805  7.8928  10.5825 -9.0837  6.9427   
3  train_3       0  11.0604 -2.1518   8.9522  7.1957  12.5846 -1.8361  5.8428   
4  train_4       0   9.8369 -1.4834  12.8746  6.6375  12.2772  2.4486  5.9405   

     var_7   ...     var_190  var_191  var_192  var_193  var_194  var_195  \
0  18.6266   ...      4.4354   3.9642   3.1364   1.6910  18.5227  -2.3978   
1  16.5338   ...      7.6421   7.7214   2.5837  10.9516  15.4305   2.0339   
2  14.6155   ...      2.9057   9.7905   1.6704   1.6858  21.6042   3.1417   
3  14.9250   ...      4.4666   4.7433   0.7178   1.4214  23.0347  -1.2706   
4  19.2514   ...     -1.4905   9.5214  -0.1508   9.1942  13.2876  -1.5121   

   var_196  var_197  var_198  var_199

## LGBM model

In [7]:
lgbmModels = []
lgbmACC = []
lgbmAUC = []

params = {
        'task': 'train',
        #'boosting_type': 'gbdt',
        'objective': 'xentropy',#'binary',xentropy;xentlambda
        'num_leaves': int(30),      # 31
        'learning_rate': np.exp(-4.108467241113353),   #0.01,
        'feature_fraction': 0.02720614438477189,#0.9,
        'bagging_fraction': 0.8350603586533163,#0.8,
        'bagging_freq': 1,
        'max_depth': 6,        #-1,
        'min_data_in_leaf': 13, #20,
        'lambda_l2': 4,        # 0,
        'is_unbalance' : False,
        'max_bin' : int(75),
        'verbose': -1
    }

In [8]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20):      
        limit = 1000 # number of 1s 
        
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
              
        
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
        
        
        lgb_train = lgb.Dataset(X_train[train_indexMod], y_train[train_indexMod]) 
        gbm = lgb.train(params,lgb_train, num_boost_round=4575, verbose_eval= 1)
        
        
        lgb_train = lgb.Dataset(X_train[train_index], y_train[train_index]) 
        gbm = lgb.train(params,lgb_train, num_boost_round=3688, verbose_eval= 1)
        y_pred = gbm.predict(X_train[test_index])
        #print('Ones', np.sum(y_pred > 0.5))
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        lgbmACC.append(accData)
        lgbmAUC.append(rocData)
        lgbmModels.append(copy.deepcopy(gbm) )
        #print('ROC rounded: ', roc_auc_score(y_train[test_index],np.round(y_pred)))
        #print('ROC proba: ', roc_auc_score(y_train[test_index],gbm.predict_proba(X_train[test_index])))
        #indices = (y_train[test_index] == 1)
        #buff = y_train[test_index] 
        #print('ACCminorClass:', np.sum( np.round(y_pred[indices])==buff[indices] )/indices.shape )
        


test 39913
n - iteration
ROC :  0.8974994680028058
ACC: 0.9202769930751731
n - iteration
ROC :  0.8977776206098823
ACC: 0.9204769880752981
n - iteration
ROC :  0.8973681798063405
ACC: 0.9203269918252044


KeyboardInterrupt: 

## GBM

In [19]:
gbmModels = []
gbmACC = []
gbmAUC = []

params = {
            'task': 'train',
            'booster': 'gbtree',
            'objective': 'binary:logistic',#'binary',
            'num_leaves': int(30),      # 31
            'learning_rate': np.exp(-4.108467241113353),   #0.01, Uses log values to improve optimization
            'colsample_bytree': 0.02720614438477189,#0.9,
            'subsample':  0.8350603586533163,#0.8,
            'max_depth': 6,        #-1,
            'min_child_weight': 10, #default=1,
            'lambda':4,        # 0,
            #'is_unbalance' : False,
            'max_bin' : int(75),
            #'nthread':3,
            #'verbosity': 0    
            'silent':1
        }

In [20]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 1000 # number of 1s 
        
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]        
        
        xgb_train = xgb.DMatrix(data = X_train[train_indexMod], label= y_train[train_indexMod]) 
        gbm = xgb.train(params,xgb_train, 4575)

        y_pred = gbm.predict(xgb.DMatrix(data =X_train[test_index]))
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        gbmACC.append(accData)
        gbmAUC.append(rocData)
        gbmModels.append(copy.deepcopy(gbm) )
        #print('ROC rounded: ', roc_auc_score(y_train[test_index],np.round(y_pred)))
        #print('ROC proba: ', roc_auc_score(y_train[test_index],gbm.predict_proba(X_train[test_index])))
        #indices = (y_train[test_index] == 1)
        #buff = y_train[test_index] 
        #print('ACCminorClass:', np.sum( np.round(y_pred[indices])==buff[indices] )/indices.shape )
        
        

test 39913
n - iteration 0
ROC :  0.8345138693293214
ACC: 0.7666058348541287
n - iteration 1
ROC :  0.8385791506047761
ACC: 0.7724806879828005
n - iteration 2
ROC :  0.8370344402331745
ACC: 0.7648058798530036
n - iteration 3
ROC :  0.8345760255447147
ACC: 0.7649058773530661
n - iteration 4
ROC :  0.8318720348674902
ACC: 0.7599810004749882
n - iteration 5
ROC :  0.8356386026566537
ACC: 0.7689057773555661
n - iteration 6
ROC :  0.8292633854158241
ACC: 0.7605059873503163
n - iteration 7
ROC :  0.8314984649858735
ACC: 0.7532061698457538
n - iteration 8
ROC :  0.8292786608908156
ACC: 0.7665058373540662
n - iteration 9
ROC :  0.8295677749215624
ACC: 0.7611809704757381
n - iteration 10
ROC :  0.8286694117583617
ACC: 0.7516062098447539


KeyboardInterrupt: 

# SVM

In [4]:
svmModels = []
svmACC = []
svmAUC = []

In [18]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 1000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
              
        
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
    
        model = LinearSVC( C =1 )
        #clf = CalibratedClassifierCV(model) 
        model.fit(X_train[train_indexMod], y_train[train_indexMod])
    
        y_pred = model.predict(X_train[test_index])
                
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        svmACC.append(accData)
        svmAUC.append(rocData)
        svmModels.append(copy.deepcopy(model) )

test 39913


NameError: name 'random' is not defined

# FM

In [43]:
fmModels = []
fmACC = []
fmAUC = []

model = TFFMClassifier(
                order=3,
                rank=10,
                optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
                n_epochs=50, 
                batch_size=1024,
                init_std=0.01,
                reg=10.0,
                input_type='dense'
                )
        

In [47]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 10000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
        
        ### improve the way I save the models
        model = TFFMClassifier(
                order=3,
                rank=10,
                optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
                n_epochs=50, 
                batch_size=1024,
                init_std=0.01,
                reg=10.0,
                input_type='dense'
                )
        
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
        
        model.fit(X_train, y_train, show_progress=False)
        y_pred = model.predict_proba(X_train[test_index])[:,1]
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        fmACC.append(accData)
        fmAUC.append(rocData)
        fmModels.append(model)
        

test 39913
n - iteration 0
ROC :  0.8632603221628441
ACC: 0.9150771230719232
n - iteration 1
ROC :  0.8654422918895421
ACC: 0.9142521436964076
n - iteration 2
ROC :  0.8644538210534276
ACC: 0.9137521561960951
n - iteration 3
ROC :  0.8642540369219188
ACC: 0.9138771530711732
n - iteration 4
ROC :  0.8641157937004066
ACC: 0.9150521236969076
n - iteration 5
ROC :  0.8653397156404132
ACC: 0.9138771530711732


KeyboardInterrupt: 

In [46]:
from sklearn.metrics import classification_report
print(classification_report(y_train[test_index], np.round(y_pred)))

             precision    recall  f1-score   support

          0       0.93      0.99      0.95     35981
          1       0.70      0.29      0.41      4020

avg / total       0.90      0.92      0.90     40001



#  Isolation forest

In [22]:
clf = IsolationForest( max_samples=10000,
                      random_state=42, contamination=0.1,n_estimators = 1000,max_features=0.1)
isoModels = []

In [24]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print(train_index[0])
    clf.fit( X_train[train_index] )
    isoModels.append(clf)
#y_pred_train = clf.predict(X_train)

39913
0
0
0
0


# Logistic Regression

In [33]:
clf = LogisticRegression(random_state=0, solver='lbfgs',C = 0.1)
logModels = []
logACC = []
logAUC = []

In [37]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 10000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
              
        
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
    
        clf.fit(X_train[train_indexMod], y_train[train_indexMod])
        
        y_pred = clf.predict_proba(X_train[test_index])[:,1]
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        logACC.append(accData)
        logAUC.append(rocData)
        logModels.append(clf)

test 39913
n - iteration 0
ROC :  0.8462581965246722
ACC: 0.7719807004824879
n - iteration 1
ROC :  0.8431798512786115
ACC: 0.7683557911052223
n - iteration 2
ROC :  0.8460713026955492
ACC: 0.770580735481613
n - iteration 3
ROC :  0.8468464630517405
ACC: 0.7711307217319567
n - iteration 4
ROC :  0.8467098237723862
ACC: 0.7699557511062224
n - iteration 5
ROC :  0.8441686332241961
ACC: 0.7666308342291442
n - iteration 6
ROC :  0.8459746029586372
ACC: 0.7699307517312067
n - iteration 7
ROC :  0.8444448707796445
ACC: 0.7637059073523161
n - iteration 8
ROC :  0.8456705038217379
ACC: 0.7680557986050349
n - iteration 9
ROC :  0.8469459420332539
ACC: 0.7743556411089723
n - iteration 10
ROC :  0.8446292895600925
ACC: 0.764980875478113
n - iteration 11
ROC :  0.8451599524403496
ACC: 0.764380890477738
n - iteration 12
ROC :  0.8433001538540034
ACC: 0.7684307892302692
n - iteration 13
ROC :  0.8441069644136395
ACC: 0.7667558311042224
n - iteration 14
ROC :  0.8439090711363557
ACC: 0.76755581110472

# Extra trees

In [43]:
model = ExtraTreesClassifier(n_estimators=1000, min_samples_split=2, random_state=0)
extModels = []
extACC = []
extAUC = []

In [44]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 10000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
        
        model.fit(X_train[train_indexMod], y_train[train_indexMod])
        
        y_pred = model.predict_proba(X_train[test_index])[:,1]
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        extACC.append(accData)
        extAUC.append(rocData)
        extModels.append(model)

test 39913
n - iteration 0
ROC :  0.8658828367265697
ACC: 0.8021799455013625


KeyboardInterrupt: 

# Random Forest

#  Autoencoders